In [1]:
from newspaper import Article
from newsapi import NewsApiClient
import pandas as pd

In [2]:
newsapi = NewsApiClient(api_key="<KEY>")

In [4]:
df1 = pd.DataFrame()
total_articles = list()
 
total_pages = 1
while total_pages<=5:
    all_data = newsapi.get_everything( q='Tesla',
                                      sources='reuters',
                                      from_param='2022-09-25',
                                      to='2022-10-13',
                                      language='en',
                                      page = total_pages                        
                                       )
     
    df2 = pd.DataFrame.from_records(all_data['articles'])
    df1 = pd.concat([df1,df2])
   
    total_pages += 1

In [5]:
print(df1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   source       93 non-null     object
 1   author       7 non-null      object
 2   title        93 non-null     object
 3   description  93 non-null     object
 4   url          93 non-null     object
 5   urlToImage   93 non-null     object
 6   publishedAt  93 non-null     object
 7   content      93 non-null     object
dtypes: object(8)
memory usage: 5.9+ KB
None


In [6]:
def get_article_text(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
    except:
        return ""
    return article.text

df1['article_text'] = df1.apply (lambda row: get_article_text(row['url']), axis=1)
df1.to_csv('../../data/raw_data.csv')

In [21]:
import sys
sys.path.append("../") # go to parent dir
from utility import nlp_text_cleaner
%load_ext autoreload
%autoreload 2

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\metes\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\metes\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [51]:
raw_data = pd.read_csv('../../data/raw_data.csv',dtype={'article_text':'string'})
raw_data.info()
raw_data = raw_data.dropna(subset=['article_text'])
raw_data.info()
paragraph_clean_data = pd.DataFrame()
paragraph_clean_data['text'] = raw_data["article_text"].apply(nlp_text_cleaner.split_into_sentences)

paragraph_clean_data = paragraph_clean_data.explode('text')

#paragraph_clean_data['text'] = paragraph_clean_data["text"].apply(nlp_text_cleaner.replace_contraction)
#paragraph_clean_data['text'] = paragraph_clean_data["text"].apply(nlp_text_cleaner.remove_unicode)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    93 non-null     int64 
 1   source        93 non-null     object
 2   author        7 non-null      object
 3   title         93 non-null     object
 4   description   93 non-null     object
 5   url           93 non-null     object
 6   urlToImage    93 non-null     object
 7   publishedAt   93 non-null     object
 8   content       93 non-null     object
 9   article_text  92 non-null     string
dtypes: int64(1), object(8), string(1)
memory usage: 7.4+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 92 entries, 0 to 92
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    92 non-null     int64 
 1   source        92 non-null     object
 2   author        6 non-null      object
 3   title         

In [52]:
paragraph_clean_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1774 entries, 0 to 92
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1774 non-null   object
dtypes: object(1)
memory usage: 27.7+ KB


In [54]:
import json
paragraph_clean_data["label"] = ""
dic = paragraph_clean_data.to_dict('records')

with open('../../data/sentences_clean_data.jsonl', 'w') as outfile:
    for entry in dic:
        json.dump(entry, outfile)
        outfile.write('\n')

In [ ]:
## Now convert this data to annotated data using doccano
## 250_sentences_annotated_data.jsonl will be used for spacy training
## format is  "text":"Tesla did not immediately respond with a comment.","label":[[0,6,"CUSTOM_ORG"]]
# Custom Entities: CUSTOM_ORG,CUSTOM_PERSON,CUSTOM_PLACE,CUSTOM_ROLE

In [69]:
###  For transformerr training we'll create BILOU format.
# Example, tokens = ['I','am','Elon']   ner_tags=['O','O',"U-PERSON"]
#For this there is no need to annotate again.
## We'll convert data used for spacy training to BILOU format using Spacy library itself.
## Custom Entities:  B-CUSTOM_ORG,I-CUSTOM_ORG,L-CUSTOM_ORG,U-CUSTOM_ORG,B-CUSTOM_PERSON,
# I-CUSTOM_PERSON,L-CUSTOM_PERSON,U-CUSTOM_PERSON,B-CUSTOM_ROLE,I-CUSTOM_ROLE,L-CUSTOM_ROLE,
# U-CUSTOM_ROLE,B-CUSTOM_PLACE',I-CUSTOM_PLACE,L-CUSTOM_PLACE,U-CUSTOM_PLACE


# Create dataset with one sample/sentence per row
import json
with open('../../data/250_sentences_annotated_data.jsonl', 'r') as file:
    labeled_data =  [json.loads(line) for line in file]



In [70]:
import spacy
from spacy.training import offsets_to_biluo_tags
nlp = spacy.load("en_core_web_lg")

In [91]:
entities = []
final_data = pd.DataFrame(columns = ['tokens','ner_tags'])
for row in labeled_data:
    #doc = nlp(row['text'])
    doc = nlp.make_doc(row['text'])
    entities = row['label']
    tokens = []
    ner_tags = []
    #iob_tags = offsets_to_biluo_tags(doc, entities)
    iob_tags = offsets_to_biluo_tags(doc, entities)
        
    for i in range(len(iob_tags)):
        tokens.append(str(doc[i].text))
        iob_tag = iob_tags[i]
        if iob_tag == "-":
            iob_tag = "O"
        ner_tags.append(iob_tag)
       
    new_row = pd.Series({'tokens': tokens, 'ner_tags': ner_tags})    
    final_data = pd.concat([final_data, new_row.to_frame().T], ignore_index=True)

c:\Users\metes\Anaconda3\envs\custom_ner_api\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "Summary

Summary Companies Tesla down as Q3 delive..." with entities "[[27, 33, 'CUSTOM_ORG'], [78, 81, 'CUSTOM_PLACE'],...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\metes\Anaconda3\envs\custom_ner_api\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "The U.S. stock market has suffered three quarterly..." with entities "[[4, 7, 'CUSTOM_ORG']]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\metes\Anaconda3\envs\custom_ner_api\lib\site-packages\spacy\training\iob_utils.py:141: UserWarni

In [92]:
final_data.iloc[0].ner_tags

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-CUSTOM_ORG',
 'L-CUSTOM_ORG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'U-CUSTOM_ORG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [93]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tokens    250 non-null    object
 1   ner_tags  250 non-null    object
dtypes: object(2)
memory usage: 4.0+ KB


In [94]:
final_data.to_csv('../../data/BIOUL_data.csv',index=False)